<a href="https://colab.research.google.com/github/Carlosvergara1995/Problem_Set_4_Predicting_Tweets/blob/main/Analizador_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##TProblem_Set_4_Predicting_Tweets


##CARGA DE DATOS

In [ ]:
#IMPORTAMOS LIBRERIAS
import pandas as pd
import numpy as np